# A notebook to email geocomp grads the kata solutions

This is based on my [email from python](Email_from_python.ipynb) notebook which is based on [Corey Schafer's video](https://www.youtube.com/watch?v=JRCJ6RtE3xU&t=751s).

For the Google API, I used both [Google's docs](https://developers.google.com/sheets/api/guides/values) and [this medium article](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c) by [Prafulla Dalvi](https://medium.com/@prafuld3).

In [1]:
#email imports
import smtplib
from email.message import EmailMessage
from enviro import GMAIL_LOGIN, GMAIL_PWD, SPREADSHEET_ID, SPREADSHEET_ID_TESTING, TEST_EMAILS, PROJECT_PATH
#Google API imports
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
#General imports
import glob
import pandas as pd
from datetime import date

In [2]:
### GOOGLE API GLOBALS
# If modifying these scopes, delete the file token.pickle and rerun pickle creation functions.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# set up variables
login = GMAIL_LOGIN
password = GMAIL_PWD

In [3]:
def connect_to_google_sheets_API():
    """
    Connect to the Google Sheet API, verify credits and return a service object.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)
    print('read service created successfully')

    return service

In [4]:
def connect_to_google_sheets_API_write():
    """
    Connect to the Google Sheet API, verify credits and return a service object.
    """
    creds = None
    if os.path.exists('token_write.pickle'):
        with open('token_write.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token_write.pickle', 'wb') as token:
            pickle.dump(creds, token)

    try:
        service = build('sheets', 'v4', credentials=creds)
        print('write service created successfully')
        return service
    except Exception as e:
        print(e)
        return None

In [5]:
def read_from_google_sheet(service, spreadsheet_id):
    """Call the Sheets API"""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range='Sheet1').execute()
    values = result.get('values', [])

    if not values:
        print('No data found, aborting.')
        return None
    else:
        df = pd.DataFrame.from_records(values, columns=values[0])
        df = df.drop([0])
        df = df.reset_index(drop=True)
    
    return df

In [6]:
def write_to_google_sheet_log_file(df, service, spreadsheet_id):
    """Write date back to Google Sheet log file for given class_abr and challenge"""
    response_date = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id,
                                                           valueInputOption='RAW',
                                                           range='Sheet1',
                                                           body=dict(
                                                               majorDimension='ROWS',
                                                               values=df.T.reset_index().T.values.tolist()
                                                           )
                                                          ).execute()
    print('Google sheet successfully updated')
    return None

In [7]:
def get_recipient_names_and_emails(class_abr, spreadsheet_id):
    """
    Given a class abreviation, return a dict of names and emails.
    args:
        class_abr: `str` class abreviation for example: `apr20` or `mar20`
    """
    
    df = read_from_google_sheet(connect_to_google_sheets_API(), spreadsheet_id)
    df = df.loc[df['class_abr'] == class_abr]
    if len(df['class_abr'] == class_abr) == 0:
        print('The `class_abr` you provided does not exist or was misspelled.')
        return None
    
    return pd.Series(df.email.values, index=df.recipient).to_dict()

In [8]:
def get_challenge_solution(challenge, project_path):
    """Given a challenge name, return the corresponding path to the solution file."""
    challenge = challenge.split('-')[0].title() + '-' + challenge.split('-')[1] if '-' in challenge else challenge.title()
        
    try:
        solution = glob.glob(f'{project_path}{challenge}_solution.ipynb')[0]
    except IndexError:
        print('The challenge you provided does not exist or was misspelled.')
        return None
    
    return solution

In [9]:
def make_df(class_abr, challenge, date, spreadsheet_id):
    df = read_from_google_sheet(connect_to_google_sheets_API(), spreadsheet_id)
    
    column = get_challenge_solution(challenge, PROJECT_PATH).split('/')[-1]
    df.loc[df['class_abr'] == class_abr, column] = str(date)
    df = df.fillna('')
    return df

In [10]:
def send_kata_solution(challenge, recipient, msg_to, msg_from='robert@agilescientific.com', run_case=0):
    """
    Send an email with the kata solution as an attachment to the recipient, by name.
    
    args:
        challenge: `str` name of the kata challenge
        recipient: `str` name of the recipient
        msg_to: `str` valid email address of the `recipient` to send the message to
    kwargs:
        msg_from: `str` email from which to send the email - this is only for the reply-to field.
            The email is actually sent from my personal-work email
        run_case: `int` from `set({0, 1, 2, 99})`where:
            0: Aborts with no action, fail safe in case this function is not run on purpose.
            1: Print out dry_run message for each email in class_abr but no emails sent,
               does not write to Google sheet
               `print('dry run, no emails sent')`
               `pass`
            3: Send email out to test_emails only, does not write to Google sheet
            99: *NOT A TEST*, this sends actual emails for each email in class_abr with attached 
                solution and *DOES WRITE TO* Google sheet
                
    Returns:
        None
    """

    msg = EmailMessage()
    msg['Subject'] = f'kata.geosci solution for challenge: {challenge}'
    msg['From'] = msg_from
    msg['To'] = msg_to
    msg['Cc'] = msg_from
    msg.set_content(f"""Dear {recipient}, 

I am enclosing the solutions to the {challenge} challenge. I would recommend that you only use this if you're really stuck, or, once you've finished, to see some insights included in the solution.

You can get the next challenge by completing this one !

I'd also like to point you towards Matt's blog about this: 
https://agilescientific.com/blog/2020/4/16/geoscientist-challenge-thyself

and remember you can also get help here https://help.agilescientific.com/home

Finally we'd love to hear your feedback about these challenges. Are they fun? are they useful? are they hard enough or too hard?

And of course, you can unsubscribe at any time by replying to me.

Best wishes,
Rob

Robert Leckenby, PhD
Geoscientist at Agile*
agilescientific.com

PS: this email was sent with Python, because why not ;-)
    """)

    if run_case == 99:
        with open(get_challenge_solution(challenge, PROJECT_PATH), 'rb') as f:
            file_data = f.read()
            file_name = f.name.split('/')[-1]

        msg.add_attachment(file_data, maintype='text', subtype='ipynb', filename=file_name)

        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(login, password)
            # *************** WARNING ***************
            # the next line will send an actual email
            smtp.send_message(msg)

        # logging
        print(f'On {date.today()} the *{challenge}* challenge was sent to:\n{msg_to}\n')
        return
        
    elif run_case == 0:
        print('Aborting with no action, fail safe in case this function was not run on purpose.\nEnd.')
        return
    
    elif run_case == 1:
        print('Dry run only, no email sent.')
        print(f'arguments passed: {recipient, msg_to, challenge}\n#################')
        return
    
    elif run_case == 3:
        with open(get_challenge_solution(challenge, PROJECT_PATH), 'rb') as f:
            file_data = f.read()
            file_name = f.name.split('/')[-1]

        msg.add_attachment(file_data, maintype='text', subtype='ipynb', filename=file_name)

        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(login, password)
            # *************** WARNING ***************
            # the next line will send an actual email
            smtp.send_message(msg)

        # logging
        print(f'On {date.today()} the *{challenge}* challenge was sent to:\n{msg_to}\n')
        return
    
    else:
        print('Incorrect `run_case` kwarg passed, `run_case` must be `int` from `set({0, 1, 3, 99})`')
        print('See docstring for details.\nEnd.')
        return
    
    return

In [11]:
def send_emails(class_abr, challenge, project_path, run_case=0):
    """
    Sends emails to students with a given challenge. Provides provision for manual tests.
    args:
        class_abr: `str` class abreviation of type "apr20" or "mar20" for example
        challenge: `str` kata challenge name
        project_path: `str` path to kata solutions folder
    kwargs:
        run_case: `int` from `set({0, 1, 2, 3, 99})` where:
            0: Dry run only, simply lists out names, emails and challenges for recipients in this class_abr, 
               does not write to Google sheet
            1: Print out dry_run message for each email in class_abr but no emails sent,
               does not write to Google sheet
            2: Print out dry_run message for each email in class_abr but no emails sent,
               *DOES WRITE TO* Google sheet TEST FILE
            3: Send email out to test_emails only, does not write to Google sheet
            99: *NOT A TEST*, this sends actual emails for each email in class_abr with attached 
                solution and *DOES WRITE TO* Google sheet PRODUCTION FILE
    """
    
    if run_case == 99:
        spreadsheet_id = SPREADSHEET_ID
        
        if (get_recipient_names_and_emails(class_abr, spreadsheet_id) == None) or (get_challenge_solution(challenge, PROJECT_PATH) == None):
            print('aborting.')
            return
            
        for recipient, email in get_recipient_names_and_emails(class_abr, spreadsheet_id).items():
            send_kata_solution(challenge, recipient, email, run_case=99)
            
        df = make_df(class_abr, challenge, date.today(), spreadsheet_id)
        service = connect_to_google_sheets_API_write()
    
        write_to_google_sheet_log_file(df, service, spreadsheet_id)
        
        print(f'Check sending email account ({login}) for errors.\n')
        print('Done =============================================\n')
        return
    
    elif run_case == 0:
        spreadsheet_id = SPREADSHEET_ID_TESTING
        print('Dry run only. Emails would have been sent to these recipients at these emails:')
        print('##############################################################################')
        for recipient, email in get_recipient_names_and_emails(class_abr, spreadsheet_id).items():
            print(f'recipient: {recipient}\temail: {email}\tchallenge: {challenge}')
        print('End.')
        return
    
    elif run_case == 1:
        spreadsheet_id = SPREADSHEET_ID_TESTING
        if (get_recipient_names_and_emails(class_abr, spreadsheet_id) == None) or (get_challenge_solution(challenge, PROJECT_PATH) == None):
            print('aborting.')
            return   
        for recipient, email in get_recipient_names_and_emails(class_abr, spreadsheet_id).items():
            send_kata_solution(challenge, recipient, email, run_case=1)
        return
    
    elif run_case == 2:
        spreadsheet_id = SPREADSHEET_ID_TESTING
        if (get_recipient_names_and_emails(class_abr, spreadsheet_id) == None) or (get_challenge_solution(challenge, PROJECT_PATH) == None):
            print('aborting.')
            return
        for recipient, email in get_recipient_names_and_emails(class_abr, spreadsheet_id).items():
            send_kata_solution(challenge, recipient, email, run_case=1)
        
        df = make_df(class_abr, challenge, date.today(), spreadsheet_id)
        service = connect_to_google_sheets_API_write()
        
        write_to_google_sheet_log_file(df, service, spreadsheet_id)
        return
    
    elif run_case == 3:
        challenge = 'test'
        emails_test = TEST_EMAILS
        for recipient, email in emails_test.items():
            send_kata_solution(challenge, recipient, email, run_case=3)
            
        print(f'Check sending email account ({login}) for errors.\n')
        print('Done =============================================\n')
        return
    
    else:
        print('Incorrect `run_case` kwarg passed, `run_case` must be `int` from `set({0, 1, 2, 3, 99})`')
        print('See docstring for details.\nEnd.')

#### Manual tests

In [12]:
send_emails('test', 'test', PROJECT_PATH, run_case=0)

Dry run only. Emails would have been sent to these recipients at these emails:
##############################################################################
read service created successfully
recipient: Robert Leckenby	email: rjleckenby@gmail.com	challenge: test
recipient: Robert [Agile]	email: robert@agilescientific.com	challenge: test
recipient: Rob [Agile]	email: rob@agilescientific.com	challenge: test
End.


In [13]:
send_emails('Hess_july', 'fossil-hunting', PROJECT_PATH, run_case=1)

read service created successfully
read service created successfully
Dry run only, no email sent.
arguments passed: ('Chance Amos', 'camos@hess.com', 'fossil-hunting')
#################
Dry run only, no email sent.
arguments passed: ('Reed Bracht', 'rbracht@hess.com', 'fossil-hunting')
#################
Dry run only, no email sent.
arguments passed: ('Peter Steele', 'psteele@hess.com', 'fossil-hunting')
#################
Dry run only, no email sent.
arguments passed: ('Julia Peacock', 'JPeacock@hess.com', 'fossil-hunting')
#################
Dry run only, no email sent.
arguments passed: ('Chase Woodward', 'ChBatchelor@hess.com', 'fossil-hunting')
#################
Dry run only, no email sent.
arguments passed: ('Estefania Ortiz', 'EOrtiz@hess.com', 'fossil-hunting')
#################
Dry run only, no email sent.
arguments passed: ('Abra Ziegler', 'AZiegler@hess.com', 'fossil-hunting')
#################
Dry run only, no email sent.
arguments passed: ('Mike Lee', 'milee@hess.com', 'fossil

In [ ]:
send_emails('test', 'test', PROJECT_PATH, run_case=2)

In [ ]:
send_emails('test', 'test', PROJECT_PATH, run_case=3)

In [ ]:
send_emails('test', 'test', PROJECT_PATH, run_case=4)

In [ ]:
send_kata_solution('test', 'Rob', 'rob@agilescientific.com', run_case=0)

In [ ]:
send_kata_solution('test', 'Rob', 'rob@agilescientific.com', run_case=123)

#### SEND KATA EMAILS

In [14]:
send_emails('Hess_july', 'fossil-hunting', PROJECT_PATH, run_case=99)

read service created successfully
read service created successfully
On 2020-09-10 the *fossil-hunting* challenge was sent to:
camos@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
rbracht@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
psteele@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
JPeacock@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
ChBatchelor@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
EOrtiz@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
AZiegler@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
milee@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
iwolfe@hess.com

On 2020-09-10 the *fossil-hunting* challenge was sent to:
RArmstrong@hess.com

read service created successfully
write service created successfully
Google sheet successfully updated
Check sending email account (fracgeol@gmail.com) for errors.

Done

#### SEND OTHER EMAILS

In [ ]:
def send_ml_online(recipient, msg_to, msg_from='robert@agilescientific.com', run_case=0):
    """
    Send an email with the Online Machine Learning details to the recipient, by name.
    
    args:
        recipient: `str` name of the recipient
        msg_to: `str` valid email address of the `recipient` to send the message to
    kwargs:
        msg_from: `str` email from which to send the email - this is only for the reply-to field.
            The email is actually sent from my personal-work email
        run_case: `int` from `set({0, 1, 2, 99})`where:
            0: Aborts with no action, fail safe in case this function is not run on purpose.
            99: *NOT A TEST*, this sends actual emails for each recipient email
                
    Returns:
        None
    """

    msg = EmailMessage()
    msg['Subject'] = f'Online Machine Learning for Subsurface'
    msg['From'] = msg_from
    msg['To'] = msg_to
    msg['Cc'] = msg_from
    msg.set_content(f"""Dear {recipient}, 

we now have the details of our Online Machine Learning class!
Course description, dates, times, prices and registration are all visible and available here:

https://agilescientific.com/online-machine-learning

And as Matt says:
"In particular, if you know how to:
- Write a function in Python.
- Load a CSV into NumPy or Pandas.
- Make a plot.
…then you should have no trouble in this class!"

So I hope to see you there and as always don't hesitate to get in touch with any questions you might have.

Best wishes,
Rob

Robert Leckenby, PhD
Geoscientist at Agile*
    """)

    if run_case == 99:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(login, password)
            # *************** WARNING ***************
            # the next line will send an actual email
            smtp.send_message(msg)

        # logging
        print(f'On {date.today()} the Online ML info was sent to:\n{msg_to}\n')
        return
        
    elif run_case == 0:
        print('Aborting with no action, fail safe in case this function was not run on purpose.\nEnd.')
        return
    
    else:
        print('Incorrect `run_case` kwarg passed, `run_case` must be `int` from `set({0, 99})`')
        print('See docstring for details.\nEnd.')
        return
    
    return

In [ ]:
def send_ML_online_emails(recipients, run_case=0):
    """
    Sends emails to students with info about ML class. Provides provision for manual tests.
    args:
        recipients: `dict` of recipients and emails to write to
    kwargs:
        run_case: `int` from `set({0, 1, 2, 3, 99})` where:
            0: Dry run only, simply lists out names, emails and challenges for recipients
            99: *NOT A TEST*, this sends actual emails for each email in recipients.items()
    """
    
    if run_case == 99:            
        for recipient, email in recipients.items():
            send_ml_online(recipient, email, run_case=99)
            
        print(f'Check sending email account ({login}) for errors.\n')
        print('Done =============================================\n')
        return
    
    elif run_case == 0:
        spreadsheet_id = SPREADSHEET_ID_TESTING
        print('Dry run only. Emails would have been sent to these recipients at these emails:')
        print('##############################################################################')
        for recipient, email in recipients.items():
            print(f'recipient: {recipient}\temail: {email}')
        print('End.')
        return
    
    else:
        print('Incorrect `run_case` kwarg passed, `run_case` must be `int` from `set({0, 1, 2, 3, 99})`')
        print('See docstring for details.\nEnd.')

In [ ]:
grads = {'Chris Braun':'chrisbraun@gmx.net',
         'Yann Chiffoleau':'yann_chiffoleau@hotmail.com',
         'JanOve Knutsen':'knutsjo@protonmail.com',
         'Eric Champod':'champod.eric@bluewin.ch',
         'Nils Oesterling':'nils.oesterling@swisstopo.ch',
         'Gwendoline Galland':'galland.gwendoline@gmail.com',
         'Vincent Divry':'vincentdivry@yahoo.com',
         'Kim Romailler':'kim@romailler.ch',
         'Loïc Godail':'loic.godail@oulhenco.com',
         'Wim Folkerts':'Wim.W.Folkerts@shell.com',
         'Lauren Chedburn':'l.chedburn.19@abdn.ac.uk',
         'Florian Smit':'florian.smit@ign.ku.dk',
         'Fabio Pallottino':'fabio.pallottini@cepsa.com',
         'Dylan Loss':'Dylan.P.Loss@conocophillips.com',
        }

In [ ]:
tests = {'Robert Leckenby': 'rjleckenby@gmail.com',
         'Rob [Agile]': 'rob@agilescientific.com'
        }

In [ ]:
send_ML_online_emails(tests)

In [ ]:
send_ML_online_emails(grads, run_case=99)